#1 거리기반 필터링 - NN(NearestNeighbors)
---
* 가장 가까운 거리에 있는 수유실을 찾아서 n개(n = 3)를 추천하는 모델
* input  
 {'nursing_room_id': [1, 2, 3, 4, 5],  
    'latitude': [37.1, 37.2, 37.3, 37.4, 37.5],  
    'longitude': [127.1, 127.2, 127.3, 127.4, 127.5],}
* output
 {



In [ ]:
import pandas as pd
import

In [ ]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# 예시 데이터셋
# columns: ['nursing_room_id', 'latitude', 'longitude', 'rating']
nursing_rooms = pd.DataFrame({
    'nursing_room_id': [1, 2, 3, 4, 5],
    'latitude': [37.1, 37.2, 37.3, 37.4, 37.5],
    'longitude': [127.1, 127.2, 127.3, 127.4, 127.5],
    'rating': [4.5, 3.0, 5.0, 4.0, 4.8]
})

# 사용자의 현재 위치
user_location = {'latitude': 37.25, 'longitude': 127.25}

# 거리 기반으로 수유실을 필터링하기 위한 NearestNeighbors 모델
knn = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(nursing_rooms[['latitude', 'longitude']])

# 사용자 위치에서 가장 가까운 3개의 수유실 찾기
distances, indices = knn.kneighbors([list(user_location.values())])

# 가까운 수유실의 인덱스 가져오기
nearby_nursing_rooms = nursing_rooms.iloc[indices[0]]

# 평점으로 상위 3개의 수유실 정렬
top_nursing_rooms = nearby_nursing_rooms.sort_values(by='rating', ascending=False).head(3)

print(top_nursing_rooms)


   nursing_room_id  latitude  longitude  rating
2                3      37.3      127.3     5.0
0                1      37.1      127.1     4.5
1                2      37.2      127.2     3.0


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


#1 거리 기반 필터링 - geopy 활용
---
* Python에서 geopy 라이브러리를 사용하여 거리를 계산하여
* python 내장 함수 sorted()로 순위를 나열하여 추천함.



In [ ]:
from geopy.distance import great_circle

# 수유실 위치와 사용자 위치의 예시
nursing_homes = [
    {'id': 1, 'name': 'Nursing Home A', 'location': (37.7749, -122.4194)},
    {'id': 2, 'name': 'Nursing Home B', 'location': (37.768, -122.422)},
    # ... 추가 수유실 정보
]

user_location = (37.7749, -122.4194)

# 거리 기반으로 수유실 정렬
def sort_by_distance(user_location, nursing_homes):
    return sorted(nursing_homes, key=lambda x: great_circle(user_location, x['location']).meters)

sorted_nursing_homes = sort_by_distance(user_location, nursing_homes)

---

#2. 평점 기반 필터링
---
* user들이 남긴 평균 평점을 python 내장 함수 sorted()로 순위를 나열하여 추천함.



In [ ]:
# 각 수유실에 대한 user들의 평균 평점
nursing_homes = [
    {'id': 1, 'name': 'Nursing Home A', 'avg_rating': 4.5},
    {'id': 2, 'name': 'Nursing Home B', 'avg_rating': 4.0},
    # ...
]

# 평점 기반으로 수유실 정렬
def sort_by_rating(nursing_homes):
    return sorted(nursing_homes, key=lambda x: x['rating'], reverse=True)

sorted_by_rating = sort_by_rating(nursing_homes)



---



#3. 리뷰 기반 텍스트 분석
---
* 리뷰 텍스트에서 감정을 분석하고 긍정적인 리뷰 비율을 계산

In [ ]:
from textblob import TextBlob

# 리뷰 데이터가 있다고 가정
nursing_homes = [
    {'id': 1, 'name': 'Nursing Home A', 'reviews': ['Great place!', 'Very caring staff.']},
    {'id': 2, 'name': 'Nursing Home B', 'reviews': ['Not very clean.', 'Could be better.']},
    # ...
]

# 리뷰의 감정 분석을 통해 긍정적인 리뷰 비율을 계산
def calculate_positive_review_ratio(nursing_home):
    positive_reviews = 0
    for review in nursing_home['reviews']:
        analysis = TextBlob(review)
        if analysis.sentiment.polarity > 0:  # 긍정적인 리뷰
            positive_reviews += 1
    return positive_reviews / len(nursing_home['reviews']) if nursing_home['reviews'] else 0

# 수유실을 긍정적인 리뷰 비율에 따라 정렬
def sort_by_positive_reviews(nursing_homes):
    for home in nursing_homes:
        home['positive_review_ratio'] = calculate_positive_review_ratio(home)
    return sorted(nursing_homes, key=lambda x: x['positive_review_ratio'], reverse=True)

sorted_by_reviews = sort_by_positive_reviews(nursing_homes)



---



#4. 협업 필터링
---
* 사용자와 수유실 간의 평점 행렬이 필요
* surprise 라이브러리 활용

In [ ]:
from surprise import Dataset, Reader, KNNBasic, accuracy
from surprise.model_selection import train_test_split

# 평점 데이터가 있다고 가정
ratings_dict = {
    'itemID': [1, 1, 2, 2, 3, 3],
    'userID': [9, 32, 2, 45, 'user', 5],
    'rating': [3, 2, 4, 3, 5, 3],
}

df = pd.DataFrame(ratings_dict)
reader = Reader(rating_scale=(1, 5))

# 데이터셋 로드
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

# 훈련셋과 테스트셋을 분리
trainset, testset = train_test_split(data, test_size=.25)

# 협업 필터링 모델을 사용
algo = KNNBasic()
algo.fit(trainset)

# 모델 평가
predictions = algo.test(testset)
accuracy.rmse(predictions)



---



#5. 콘텐츠 기반 추천
---
* 수유실의 특성을 기반으로 추천하는 모델
* 각 수유실의 특성 벡터를 만들고, 사용자의 과거 평점을 기반으로 추천

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 수유실의 특성을 문자열로 나타냄
nursing_homes = [
    {'id': 1, 'name': 'Nursing Home A', 'features': 'clean comfortable'},
    {'id': 2, 'name': 'Nursing Home B', 'features': 'cheap spacious'},
    #
]

# TF-IDF 벡터화
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform([x['features'] for x in nursing_homes])

# 사용자가 과거에 평점을 높게 준 수유실의 특성을 기반으로 추천
user_profile = tfidf.transform(['clean comfortable spacious'])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(user_profile, tfidf_matrix)

# 가장 유사도가 높은 수유실을 추천
recommended_nursing_homes = cosine_sim.argsort()[0][-3:][::-1]

for idx in recommended_nursing_homes:
    print(nursing_homes[idx])




---



#6. 기계학습 모델
---
* 기계학습 모델을 사용하여 사용자의 행동을 학습하고 추천을 수행
* RandomForestClassifier모델 활용

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 특성과 라벨이 있다고 가정
features = [[0, 1], [1, 1], [1, 0], [0, 0]]  # 예: [평점이 높은가, 긍정적 리뷰가 많은가]
labels = [1, 1, 0, 0]  # 예: 사용자가 해당 수유실을 좋아할 것인가

# 랜덤 포레스트 분류기 생성
clf = RandomForestClassifier(n_estimators=10)
clf.fit(features, labels)

# 새로운 수유실에 대한 예측
new_nursing_home = [[0, 1]]  # 예: 평점은 낮지만 긍정적 리뷰가 많음
print(clf.predict(new_nursing_home))



---



#7. 하이브리드 시스템
---
* 거리 기반 필터링으로 수유실 목록을 좁힌 후, 평점과 리뷰 기반의 점수를 결합
* 거리 계산, 평점 및 리뷰 분석 등의 기능이 필요
* 사용자의 현재 위치, 수유실의 위치와 평점, 그리고 리뷰의 감정 분석 결과를 기반으로 추천을 수행

In [1]:
!pip install geopy numpy textblob

In [2]:
from geopy.distance import great_circle
import numpy as np
from textblob import TextBlob

# 예제 데이터
# 사용자의 현재 위치
current_location = (37.7749, -122.4194)

# 수유실 목록: 각각의 위치, 평점, 리뷰
nursing_rooms = [
    {'name': 'Room A', 'location': (37.7749, -122.431297), 'rating': 4.5, 'reviews': ['Clean and comfortable', 'Spacious']},
    {'name': 'Room B', 'location': (37.768, -122.422), 'rating': 4.0, 'reviews': ['A bit crowded', 'Good service']},
    {'name': 'Room C', 'location': (37.761, -122.435), 'rating': 3.5, 'reviews': ['Small but clean', 'Not very comfortable']},
    # 더 많은 수유실 데이터가 있다고 가정
]

# 거리 계산 함수
def calculate_distance(location1, location2):
    return great_circle(location1, location2).kilometers

# 평점 계산 함수
def calculate_rating(ratings):
    return np.mean(ratings)

# 감정 분석 함수
def analyze_sentiments(reviews):
    sentiments = [TextBlob(review).sentiment.polarity for review in reviews]
    return np.mean(sentiments)

# 하이브리드 추천 시스템
def hybrid_recommendation(current_location, nursing_rooms):
    recommendations = []
    for room in nursing_rooms:
        distance = calculate_distance(current_location, room['location'])
        rating_score = calculate_rating([room['rating']])
        sentiment_score = analyze_sentiments(room['reviews'])
        # 가중치는 여러분의 요구 사항에 따라 조정할 수 있음
        distance_weight = 1.0
        rating_weight = 1.0
        sentiment_weight = 1.0
        # 거리는 가까울수록 좋으므로 거리의 역수를 사용
        score = (rating_weight * rating_score + sentiment_weight * sentiment_score) / (distance_weight * distance)
        recommendations.append((room['name'], score))

    # 점수가 높은 순서로 정렬
    recommendations.sort(key=lambda x: x[1], reverse=True)
    return recommendations

# 추천 시스템 실행
recommendations = hybrid_recommendation(current_location, nursing_rooms)
for recommendation in recommendations:
    print(recommendation)


('Room B', 5.433716853681635)
('Room A', 4.486878319482427)
('Room C', 1.6708159550985253)




---



#3. 리뷰 기반 텍스트 분석
---
* 리뷰 텍스트에서 감정을 분석하고 긍정적인 리뷰 비율을 계산



---

